In [1]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from scipy import ndimage, misc
import tensorflow as tf
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from os import listdir
from os.path import isfile, join
import gc
from astropy.io import fits
import matplotlib.pyplot as plt
import random
from numpy.random import default_rng
import numpy as np
from tensorflow.keras import datasets, layers, models
rng = default_rng(42)
import glob
import seaborn as sns
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import h5py
from keras.callbacks import EarlyStopping
#from mlxtend.plotting import plot_confusion_matrix

2024-05-31 08:08:33.131066: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 08:08:33.165782: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 08:08:33.165804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 08:08:33.166868: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-31 08:08:33.172282: I tensorflow/core/platform/cpu_feature_guar

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
tf.config.list_physical_devices('GPU')


2024-05-31 08:08:35.708739: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2024-05-31 08:08:35.718060: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-31 08:08:35.718147: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [3]:
carpetas = ["z_cero",'zvar','fits_inclinated']
path = '/project/Data/SAD_CNN/'

In [5]:
with h5py.File(path + carpetas[1] + "/" + carpetas[1] + "_dataset.h5", 'r') as hdf:
    x_train_1 = hdf['x_train'][:].astype(np.float32)
    train_labels_1 = hdf['train_labels'][:].astype(int)
    x_val_1 = hdf['x_val'][:].astype(np.float32)
    val_labels_1 = hdf['val_labels'][:].astype(int)

In [6]:
with h5py.File(path + carpetas[0] + "/" + carpetas[0] + "_dataset.h5", 'r') as hdf:
    x_train_0 = hdf['x_train'][:].astype(np.float32)
    train_labels_0 = hdf['train_labels'][:].astype(int)
    x_val_0 = hdf['x_val'][:].astype(np.float32)
    val_labels_0 = hdf['val_labels'][:].astype(int)

In [7]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()


In [8]:
import h5py

# Abrir el archivo en modo de solo lectura
with h5py.File(path + carpetas[2] + "/" + carpetas[2] + "_dataset.h5", 'r') as hdf:

    train_gen = train_datagen.flow(
         np.concatenate([x_train_0,x_train_1,hdf['x_train'][:].astype(np.float32)], axis=0),
        np.concatenate([train_labels_0,train_labels_1,hdf['train_labels'][:].astype(int)], axis=0),
    batch_size=64,shuffle=True
    )
    
    val_gen = validation_datagen.flow(
    np.concatenate([x_val_0,x_val_1,hdf['x_val'][:].astype(np.float32)], axis=0),
     np.concatenate([val_labels_0,val_labels_1,hdf['val_labels'][:].astype(int)], axis=0),
    batch_size=32,shuffle=True
    )

    


In [10]:
base_model = keras.applications.Xception(
weights='imagenet',  # Load weights pre-trained on ImageNet.
input_shape=(300, 300, 3),
include_top=False)
base_model.trainable = False
inputs = keras.Input(shape=(300, 300, 3))
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1,activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
          loss=tf.keras.losses.BinaryCrossentropy(),
          metrics=[keras.metrics.BinaryAccuracy()])

model.load_weights(f'labels/{carpetas[1]}/checkpoints_transfer/my_checkpoint_transfer')

2024-05-31 08:15:49.756136: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-31 08:15:49.756285: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-31 08:15:49.756382: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [11]:
# Definir el callback de EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitorea la pérdida de validación
    patience=20,         # Número de épocas sin mejoría para detener el entrenamiento
    verbose=1,           # Mensajes de salida en el entrenamiento
    restore_best_weights=True,
    min_delta=0.001# Restaura los pesos del mejor modelo al final del entrenamiento
)

# Entrenar el modelo con el callback de EarlyStopping
history = model.fit(
    train_gen,
    epochs=200,
    validation_data=val_gen,
    callbacks=[early_stopping]  # Pasar el callback de EarlyStopping
)

history_df = pd.DataFrame(history.history)
history_df.to_csv(f"labels/{carpetas[2]}/history_trasfer.csv",index=False)
model.save_weights(f'labels/{carpetas[2]}/checkpoints_transfer/my_checkpoint_transfer')

Epoch 1/200


2024-05-31 08:16:16.566617: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906


  1/237 [..............................] - ETA: 25:03 - loss: 0.2704 - binary_accuracy: 0.8750

2024-05-31 08:16:21.154320: I external/local_xla/xla/service/service.cc:168] XLA service 0x739b939b40e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-31 08:16:21.154340: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-05-31 08:16:21.165801: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1717143381.240755  150281 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


237/237 [==============================] - 86s 336ms/step - loss: 0.3633 - binary_accuracy: 0.8522 - val_loss: 0.3076 - val_binary_accuracy: 0.8647
Epoch 2/200
237/237 [==============================] - 72s 305ms/step - loss: 0.3331 - binary_accuracy: 0.8594 - val_loss: 0.2934 - val_binary_accuracy: 0.8738
Epoch 3/200
237/237 [==============================] - 73s 306ms/step - loss: 0.3213 - binary_accuracy: 0.8615 - val_loss: 0.2866 - val_binary_accuracy: 0.8752
Epoch 4/200
237/237 [==============================] - 72s 304ms/step - loss: 0.3139 - binary_accuracy: 0.8656 - val_loss: 0.2822 - val_binary_accuracy: 0.8747
Epoch 5/200
237/237 [==============================] - 73s 309ms/step - loss: 0.3084 - binary_accuracy: 0.8683 - val_loss: 0.2792 - val_binary_accuracy: 0.8779
Epoch 6/200
237/237 [==============================] - 73s 307ms/step - loss: 0.3041 - binary_accuracy: 0.8703 - val_loss: 0.2770 - val_binary_accuracy: 0.8770
Epoch 7/200
237/237 [==============================]